In [1]:
%pylab inline
from __future__ import print_function, division
from tqdm import tqdm
import time,wave,threading,datetime
import alsaaudio, time, audioop, sys, librosa,json
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pypot.dynamixel
import pypot.robot
from explauto.environment import environments



my_config ={}
my_config['controllers'] = {}
my_config['controllers']['upper_body_controler'] = {
    'port': '/dev/ttyUSB0',     # For Linux
    #'port': '/dev/tty.usbserial-AI03QEN0', # For OSX
    'sync_read': False,
    'attached_motors': ['arms'],
    'protocol': 1,
}
my_config['motorgroups']={}
my_config['motorgroups'] = {
    'arms': ['right_arm'],
    'right_arm': ['r_shoulder_x', 'r_shoulder_y', 'r_arm_z','r_elbow_y']
}
my_config['motors'] = {}
my_config['motors']['r_shoulder_y'] = {
    'id': 51,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (-50, 170),
}
my_config['motors']['r_shoulder_x'] = {
    'id': 52,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (-90, 0),
}
my_config['motors']['r_arm_z'] = {
    'id': 53,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (-20, 95),
}
my_config['motors']['r_elbow_y'] = {
    'id': 54,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (0, 130),
}

poppy = pypot.robot.from_config(my_config)

poppy.start_sync()
time.sleep(2)

for m in poppy.motors:
    m.compliant = False
    m.goto_behavior = 'minjerk'
pos = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []


# activate microphone - A Logitech webcam
inp = alsaaudio.PCM(alsaaudio.PCM_CAPTURE,alsaaudio.PCM_NONBLOCK,device='sysdefault:CARD=C920')

inp.setchannels(2)
inp.setrate(88200)
inp.setformat(alsaaudio.PCM_FORMAT_GSM)

# The period size controls the internal number of frames per period.
# The significance of this parameter is documented in the ALSA api.
# For our purposes, it is suficcient to know that reads from the device
# will return this many frames. Each frame being 2 bytes long.
# This means that the reads below will return either 320 bytes of data
# or 0 bytes of data. The latter is possible because we are in nonblocking
# mode.
inp.setperiodsize(160)

spike = [[],[]]
data_table = []



Populating the interactive namespace from numpy and matplotlib


In [2]:
# Primitive actions

def rest_position():
    poppy.r_shoulder_x.goal_position = 0
    poppy.r_shoulder_y.goal_position = 0
    poppy.r_arm_z.goal_position = 0
    poppy.r_elbow_y.goal_position = 0

def app():
    pos.append(poppy.r_shoulder_x.present_position)
    pos.append(poppy.r_shoulder_y.present_position)
    pos.append(poppy.r_arm_z.present_position)
    pos.append(poppy.r_elbow_y.present_position)

    sys_load.append(poppy.r_shoulder_x.present_load)
    sys_load.append(poppy.r_shoulder_y.present_load)
    sys_load.append(poppy.r_arm_z.present_load)
    sys_load.append(poppy.r_elbow_y.present_load)
    
    speed.append(poppy.r_shoulder_x.present_speed)
    speed.append(poppy.r_shoulder_y.present_speed)
    speed.append(poppy.r_arm_z.present_speed)
    speed.append(poppy.r_elbow_y.present_speed)
    
    temp.append(poppy.r_shoulder_x.present_temperature)
    temp.append(poppy.r_shoulder_y.present_temperature)
    temp.append(poppy.r_arm_z.present_temperature)
    temp.append(poppy.r_elbow_y.present_temperature)
     
    volt.append(poppy.r_shoulder_x.present_voltage)
    volt.append(poppy.r_shoulder_y.present_voltage)
    volt.append(poppy.r_arm_z.present_voltage)
    volt.append(poppy.r_elbow_y.present_voltage)
    
def hand_wave(sec=10, rest=0.5):
    t0 = time.time()
    while True:
        
        t1 = time.time()
        if t1-t0 >= sec:
            break

        # run for sec
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = -70
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -170  
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = 60;
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 125
        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
    
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = -50
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -135
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = 75
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 90
        
        app()
        TIME.append(t1-t0)

        time.sleep(rest)
        app()

        

def rattle_shake(sec=20, rest=.6):
    
    shoulder_y0 = -45
    shoulder_x0 = 0
    arm_0 = 0
    elbow_0 = -65
    
    shoulder_y1 = -30
    shoulder_x1 = 0
    arm_1 = 0
    elbow_1 = -35
    
    
    poppy.r_shoulder_y.moving_speed = abs(shoulder_y1-shoulder_y0) / rest
    poppy.r_shoulder_x.moving_speed = abs(shoulder_x1-shoulder_x0) / rest
    poppy.r_arm_z.moving_speed = abs(arm_1-arm_0) / rest
    poppy.r_elbow_y.moving_speed = abs(elbow_1-elbow_0) / rest

    t0 = time.time()
    while True:
              
        t1 = time.time()
        if t1-t0 >= sec: #sec
            break
            
        

        # run for sec
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y0 #-45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x0 #5
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_0 #20
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_0 #-129
        

        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y1 #-25
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x1  #0
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_1; #0
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_1 #-10
        

  
        
        
       
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        
        app()
        TIME.append(t1-t0)
    

    
    
    for x in poppy.motors:
            x.moving_speed = 90


In [3]:
#This script is used to move the poppy arm into anatomical poses.
#Pictures will be edited for an anatomical reference.
def anatomical(rest=3):
        time.sleep(rest)
#Lateral View Pictures
#Figure 1
    #Motor 51 Shoulder Flexion
        poppy.r_shoulder_y.goal_position = 49 
        time.sleep(rest)
        #Rest Position
        rest_position()
    #Motor 51 Shoulder Extension
        poppy.r_shoulder_y.goal_position = -169 
        time.sleep(rest)
        #Rest Position
        rest_position()
#Figure 2
    #Motor 54 Elbow Extension
        poppy.r_elbow_y.goal_position = 0 
        time.sleep(rest)
        #Rest Position
        rest_position()
    #Motor 54 Elbow Flexion
        poppy.r_elbow_y.goal_position = -129
        time.sleep(rest)
        #Rest Position
        rest_position()
            
#Anterior View Pictures
#Figure 3
    #Motor 53 Pronation
        poppy.r_arm_z.goal_position = -20 
        time.sleep(rest)
        #Rest Position
        rest_position()
    #Motor 53 Supination
        poppy.r_arm_z.goal_position = 94 
        time.sleep(rest)
        #Rest Position
        rest_position()
#Figure 4       
    #Motor 52 Adduction
        poppy.r_shoulder_x.goal_position = 0
        time.sleep(rest)
        #Rest Position
        rest_position()
    #Motor 52 Abduction    
        poppy.r_shoulder_x.goal_position = 89
        time.sleep(rest)
        #Rest Position
        rest_position()



In [4]:
# audio recording setup

def outFile():
    return "../out_Data/rattle_{}.wav".format(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H:%M:%S'))

In [5]:
rest_position()
data_table=[]
spike = [[],[]]

In [6]:
def alsa_rattle():
    w = wave.open(outFile(),'w')
    # Open the device in nonblocking capture mode. The last argument could
    # just as well have been zero for blocking mode. Then we could have
    # left out the sleep call in the bottomvim of the loop

    w.setnchannels(2)
    w.setsampwidth(2)
    w.setframerate(44100)
    
    total = 0
    
    t = threading.Thread(target=rattle_shake)
    t.start()
    timeStart = time.time()
    while t.is_alive():
        timeStop = time.time()
        l,data = inp.read()
        if l:
            # Return the maximum of the absolute value of all samples in a fragment.
            frame = audioop.max(data,2)
            data_table.append(frame)
            total += frame
            ave = total / len(data_table)
            if(len(data_table) > 25) :
                if (frame > ave * 1.34):
                    spike[0].append(timeStop - timeStart)
                    spike[1].append(frame)
            w.writeframes(data)
            # print audioop.max(data, 2)
        time.sleep(.001)
    rest_position()

In [7]:
alsa_rattle()
rest_position()

In [24]:
for x in data_table:
	print(x)

In [25]:
for x in poppy.motors:
    print(x.moving_speed)

90
90
90
90


In [105]:
poppy.r_shoulder_y.goal_position = -45 #-10
poppy.r_shoulder_x.goal_position = 5  #0
poppy.r_arm_z.goal_position = 20; #0
poppy.r_elbow_y.goal_position = -65 #0

In [104]:
poppy.r_shoulder_y.goal_position = -30 #-10
poppy.r_shoulder_x.goal_position = 0  #0
poppy.r_arm_z.goal_position = 0; #0
poppy.r_elbow_y.goal_position = -35 #0

In [6]:
rattle_shake(sec=4)
rest_position()

In [9]:
# RUN at the end of working

pos = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []
data_table = []
for m in poppy.motors:
    m.compliant = True
time.sleep(1)
poppy.stop_sync()
poppy.close()

w.close()